In [7]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

In [8]:
data = pd.read_csv('../data/labeled_post_cleaned.csv')

In [14]:
import pandas as pd

df_stats_nv = pd.DataFrame()
df_stats_v = pd.DataFrame()

# Title length calculations
df_stats_nv['title_char_len'] = data[data['market_viability'] == 0]['title'].str.len()
df_stats_nv['title_word_len'] = data[data['market_viability'] == 0]['title'].str.split().str.len()
df_stats_nv['body_char_len'] = data[data['market_viability'] == 0]['body'].str.len()
df_stats_nv['body_word_len'] = data[data['market_viability'] == 0]['body'].str.split().str.len()

df_stats_v['title_char_len'] = data[data['market_viability'] == 1]['title'].str.len()
df_stats_v['title_word_len'] = data[data['market_viability'] == 1]['title'].str.split().str.len()
df_stats_v['body_char_len'] = data[data['market_viability'] == 1]['body'].str.len()
df_stats_v['body_word_len'] = data[data['market_viability'] == 1]['body'].str.split().str.len()

# Function to summarize lengths
def summary_stats(series):
    return {
        "min": series.min(),
        "max": series.max(),
        "mean": series.mean(),
        "median": series.median()
    }

def display_stats(stats):
    for k, v in stats.items():
        print(f"{k}:")
        for stat, val in v.items():
            print(f"  {stat}: {val:.2f}" if isinstance(val, float) else f"  {stat}: {val}")
# Generate statistics
stats = {
    "title_char_len_not_viable": summary_stats(df_stats_nv['title_char_len']),
    "title_char_len_viable": summary_stats(df_stats_v['title_char_len']),
    "title_word_len_not_viable": summary_stats(df_stats_nv['title_word_len']),
    "title_word_len_viable": summary_stats(df_stats_v['title_word_len']),
    "body_char_len_not_viable": summary_stats(df_stats_nv['body_char_len']),
    "body_char_len_viable": summary_stats(df_stats_v['body_char_len']),
    "body_word_len_not_viable": summary_stats(df_stats_nv['body_word_len']),
    "body_word_len_viable": summary_stats(df_stats_v['body_word_len']),
}

display_stats(stats)

title_char_len_not_viable:
  min: 2
  max: 277
  mean: 53.54
  median: 47.00
title_char_len_viable:
  min: 10
  max: 134
  mean: 57.99
  median: 55.00
title_word_len_not_viable:
  min: 1
  max: 49
  mean: 9.33
  median: 8.00
title_word_len_viable:
  min: 1
  max: 25
  mean: 9.65
  median: 9.00
body_char_len_not_viable:
  min: 14
  max: 37482
  mean: 3210.67
  median: 1495.00
body_char_len_viable:
  min: 177
  max: 19553
  mean: 2656.99
  median: 1576.00
body_word_len_not_viable:
  min: 2
  max: 6637
  mean: 551.51
  median: 261.00
body_word_len_viable:
  min: 34
  max: 3153
  mean: 436.29
  median: 266.00


In [15]:
# Title length calculations
df_stats_nv['title_num_question_marks'] = data[data['market_viability'] == 0]['title'].str.count(r'\?')
df_stats_nv['body_num_question_marks'] = data[data['market_viability'] == 0]['body'].str.count(r'\?')

df_stats_v['title_num_question_marks'] = data[data['market_viability'] == 1]['title'].str.count(r'\?')
df_stats_v['body_num_question_marks'] = data[data['market_viability'] == 1]['body'].str.count(r'\?')

# Generate statistics
stats = {
    "title_num_question_marks_not_viable": summary_stats(df_stats_nv['title_num_question_marks']),
    "body_num_question_marks_not_viable": summary_stats(df_stats_nv['body_num_question_marks']),

    "title_num_question_marks_viable": summary_stats(df_stats_v['title_num_question_marks']),
    "body_num_question_marks_viable": summary_stats(df_stats_v['body_num_question_marks']),
}

display_stats(stats)

title_num_question_marks_not_viable:
  min: 0
  max: 2
  mean: 0.24
  median: 0.00
body_num_question_marks_not_viable:
  min: 0
  max: 51
  mean: 2.35
  median: 1.00
title_num_question_marks_viable:
  min: 0
  max: 2
  mean: 0.26
  median: 0.00
body_num_question_marks_viable:
  min: 0
  max: 16
  mean: 2.15
  median: 1.00


In [ ]:
fdf_stats_nv['title_num_question_words'] = data[data['market_viability'] == 0]['title'].str.count(r'\?')
df_stats_nv['body_num_question_words'] = data[data['market_viability'] == 0]['body'].str.count(r'\?')

df_stats_v['title_num_question_words'] = data[data['market_viability'] == 1]['title'].str.count(r'\?')
df_stats_v['body_num_question_words'] = data[data['market_viability'] == 1]['body'].str.count(r'\?')

stats = {
    "title_num_question_words_not_viable": summary_stats(df_stats_nv['title_num_question_words']),
    "body_num_question_words_not_viable": summary_stats(df_stats_nv['body_num_question_words']),

    "title_num_question_words_viable": summary_stats(df_stats_v['title_num_question_words']),
    "body_num_question_words_viable": summary_stats(df_stats_v['body_num_question_words']),
}


display_stats(stats)